# Amplitude Amplification Algorithm

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

In [ ]:
from QuantumMultiplexors_Module_examples import expectation_loading_data
from PhaseAmplification_Module import load_q_gate

## 1. Data Discretization

First we create the discretized probability $p(x)$ and function $f(x)$ needed for calculating expected value $E_{x\sim p}(f)$

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 10
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## 2. $\theta$ estimation problem

After the loading steps our quantum state is: 

$$|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$$

The amplitude of the $|\Psi_{1}\rangle$ state is realted with $E_{x\sim p}(f)$ by:

$$\sin \theta = \sqrt{\sum_{x=0}^{2^-1} p(x)f(x)}$$

The main idea in the amplitude amplification is use a similar procedure like in Groover algorithm in order to increase the probability of the $|\Psi_{1}\rangle$ state. This can be in a straightoforward way by applying operator $\hat{Q}$ $k$ times:

$$\hat{Q}^{K}|\Psi\rangle = \sin{((2*K+1)\theta)}|\Psi_{1}\rangle+\cos{((2*K+1)\theta})|\Psi_{0}\rangle$$

The idea is apply K in such a way that the probability of $|\Psi_{1}\rangle$ is maximized. For this we can prepare the system $n$ times (each time with the same $K$), measure the final state and get the probabilities for measuring  $|\Psi_{1}\rangle$ y $|\Psi_{1}:

* $P_{|\Psi_{1}\rangle}$: Probability of get state $|\Psi_{1}\rangle$ (we desire a high probability here)
* $P_{|\Psi_{0}\rangle}$: Probability of get state $|\Psi_{0}\rangle$ (we desire a low probability here)

We know that this proabilities are related withe the corresponding amplitudes so: 

$$\sin^{2}((2*K+1)\theta) = P_{|\Psi_{1}\rangle}$$
$$\sin((2*K+1)\theta) = \sqrt{P_{|\Psi_{1}\rangle}}$$
$$(2*K+1)\theta = \arcsin{\sqrt{P_{|\Psi_{1}\rangle}}}$$

So in this moment, theoretically, we have solved the problem because we can calculate $\theta$ and the desired $E_{x\sim p}(f)$. But we have a practical important problem: **arcsin** is a multivaluated function and usually software packages give a solution in the $[\frac{-\pi}{2}, \frac{\pi}{2}]$. If the rotation resulting of apply $\hat{Q}^{K}$ is bigger than $\frac{\pi}{2}$ then we cannot compute properly $(2*K+1)\theta$.

This can be seen in the following cells

In [ ]:
#Creation of the complete Quntum Circuit
K=2
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
Q_Gate = load_q_gate(P_Gate, R_gate)
qbits = Qprog.registers
for k in range(K):
    Qprog.apply(Q_Gate, qbits)

circuit = Qprog.to_circ(submatrices_only=True)
job = circuit.to_job(qubits=[n_qbits])

In [ ]:
%qatdisplay circuit

In [ ]:
#Getting the final state
result = run_job(lineal_qpu.submit(job))
Phi = postprocess_results(result)

In [ ]:
Phi

Now we calculate:  
$$\theta = \frac{\arcsin{\sqrt{P_{|\Psi_{1}\rangle}}}}{(2*K+1)}$$

In [ ]:
#This is the angle asociated to the Phi state
theta_K = np.arcsin((Phi['Probability'].iloc[1])**0.5)
print('theta_K: {}'.format(theta_K))
thetaFromK = theta_K/(2*K+1)
print('thetaFromK: {}'.format(thetaFromK))

Additionally in this case we know the true value of $\theta$:

$$ \theta_{th} = \arcsin{\sqrt{\sum_{x=0}^{2^-1} p(x)f(x)}}$$

This is done in following cell

In [ ]:
theoric_theta = np.arcsin(np.sqrt(sum(p_X*f_X)))
print('theoric_theta: {}'.format(theoric_theta))

print('thetaFromK == theoric_theta? {}'.format(np.isclose(thetaFromK, theoric_theta)))

As can be seen in befor cell the calculated $\theta$ is different from $\theta_{th}$. But we can compute the probability associated with this $\theta_{th}$ using:

$$\sin^2((2*K+1)\theta_{th}$$

In [ ]:
print('|Psi_1> from  theoric_theta: {}'.format(np.sin((2*K+1)*theoric_theta)**2))
print('|Psi_1> from  Q^KPhi Probability: {}'.format(Phi['Probability'].iloc[1]))
print('Probability from Q^Phi and from theoric_theta are the same? {}'.format(
    np.isclose(np.sin((2*K+1)*theoric_theta)**2, Phi['Probability'].iloc[1])

))

So probabilities from $\hat{Q}^{K}|\Psi\rangle$ and from $\theta_{th}$ are the same but we cannot calculates the correct angle from the first because of multivalued $\arcsin$

## 3. Quantum Fourier Transformation

In order to solve the before section problem we can use the **Phase Estimation Algorithm**. 

Our operator $\hat{Q}$ applies a rotation of an angle $\theta$ on the state $|\Psi\rangle$. This operator is unitary and have 2 eigenvalues in form of phases: $\lambda_{\pm}=e^{\pm i2\theta}$. The **Phase Estimation Algorithm** allow us get this phase $\theta$.

For this we need to create n auxiliary qbits in state $|+\rangle$. Each qbit will be the controlled qbit for a controlled application of $\hat{Q}^{K}$ (for each controlled qbit the K will change). Finally over the auxiliary qbits we apply an inverse of the Quantum Fourier Operator ($\hat{Q}_{FT}$). The measurmente of this auxiliary qbits will give us an estimation of the desired $\theta$. This estimation will be more exact when more auxiliary qbits we use for the $\hat{Q}_{FT}$.

For this algorithm if we have $m$ auxiliary qbits and the measurment of the $\hat{Q}_{FT}$ is the integer $M$ then:

$$\theta = \frac{M\pi}{2^m}$$


In [ ]:
#Complete Phase Estimate algorithm
from qat.lang.AQASM import H
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
qbits = Qprog.registers[0]
Q_Gate = load_q_gate(P_Gate, R_gate)
n_aux = 5
qAux = Qprog.qalloc(n_aux)
#Controlled Q using  auxiliary qbits
for i, aux in enumerate(qAux):
    Qprog.apply(H, aux)
    for _ in range(2**i):
        Qprog.apply(Q_Gate.ctrl(), aux, qbits)
#Inverse of the Quantum Fourier Transformation        
from qat.lang.AQASM.qftarith import QFT
Qprog.apply(QFT(n_aux).dag(), qAux)
circuit = Qprog.to_circ(submatrices_only=True)

In [ ]:
%qatdisplay circuit

In [ ]:
#Posible results
job = circuit.to_job(qubits=qAux)
result = run_job(lineal_qpu.submit(job))
Phi = postprocess_results(result)
#Each posible state is an integer M and we can compute its associated theta
Phi['Thetas'] = [m*np.pi/(2**n_aux) for m in list(Phi.index)] 
Phi.sort_values('Probability', ascending=False, inplace = True)

In [ ]:
#Now we can plot te probability for each posible theta
%matplotlib inline
plt.plot(Phi['Thetas'], Phi['Probability'], 'o')


As can be seen the QPA give us two maximums of probability each one correspond to the $\pm\theta$
In order to see this we can use following cell:

In [ ]:
theta1 = Phi.iloc[0]['Thetas']
theta2 = Phi.iloc[1]['Thetas']
print(theta1, theta2)

Instead of get $\pm\theta$ we obtain ($\theta$ and $\pi-\theta$) but the result is equivalent.

In [ ]:
np.isclose(np.sin(theta1), -np.sin(theta2-np.pi))

In [ ]:
np.isclose(np.cos(theta1), np.cos(theta2-np.pi))

In [ ]:
theoric_theta = np.arcsin(np.sqrt(sum(p_X*f_X)))
print('theoric_theta: {}'.format(theoric_theta))
print('theta_1: {}'.format(theta1))
print('Theorical Integration: {}'.format(sum(p_X*f_X)))
print('Amplitude Amplification Integral: {}'.format(np.sin(theta1)**2))

If we use 10 qbits for codifying data (11 qbits circuit). 

With 5 auxiliary qbit:

* Theorical Integration: 0.6697327210971238
* Amplitude Amplification Integral: 0.6913417161825449

For 10 auxiliary qbits: 
* Theorical Integration: 0.6697327210971238
* Amplitude Amplification Integral: 0.66844492669611

When more auxiliar qbits are used the error of the integral is lower